# Project Overview: Retrieval-Augmented Generation (RAG)

## What is RAG?
Retrieval-Augmented Generation (RAG) is an advanced AI framework that combines information retrieval with natural language generation. By integrating a vector database with a generative language model, RAG enhances the contextual relevance and accuracy of generated responses.

## How Does This Project Use RAG?
This project demonstrates the power of RAG by:
1. Storing car descriptions as vector embeddings in a vector database (ChromaDB).
2. Querying the database to retrieve the most relevant information based on user queries.
3. Generating enriched and personalized responses using GPT-3.5.
4. Enhancing engagement by visualizing results with realistic AI-generated images.

## Key Features:
- **Vector Database Integration**: Efficient storage and retrieval of car metadata.
- **LLM-Driven Enrichment**: Personalized and human-like messages for recommendations.
- **Image Generation**: Realistic car images for an engaging user experience.
- **Retry Mechanism**: Robust reranking ensures reliable and context-aware outputs.

-------------

# Step 1: Setting Up the Environment

In this step, we import the necessary libraries and configure the environment for our project. This includes tools for embedding generation, vector database storage, and API interactions.

### Why Set Up the Environment?
1. **Foundation for the Project**: Ensures all necessary tools and dependencies are available for subsequent steps.
2. **Efficient Workflow**: Proper setup saves time and prevents errors during execution.

### Workflow
- Import required libraries like `openai`, `chromadb`, and `dotenv`.
- Configure environment variables to securely manage sensitive information like API keys.
- Ensure all dependencies are installed.

### Code Highlights
- **Library Imports**: Includes modules for embedding generation and vector storage.
- **Environment Configuration**: Uses `dotenv` to securely load environment variables.



In [1]:
import os
import openai
import chromadb
import numpy as np
import pandas as pd
from chromadb.utils import embedding_functions
from IPython.display import Image, display
from dotenv import load_dotenv

load_dotenv()

# Let's use Chroma to implement an in-memory vector store.
# This example will use generated data about cars for embedding storage and retrieval.


True

# Step 2: Configuring OpenAI API Key

This step ensures that the OpenAI API key is correctly configured, enabling us to access embedding and text generation functionalities.

### Why Configure an API Key?
1. **Authentication**: Required to interact with OpenAI’s API.
2. **Secure Access**: Protects sensitive credentials using environment variables.

### Workflow
- Load the OpenAI API key from environment variables using `dotenv`.
- Assign the API key to the `openai.api_key` variable.

### Code Highlights
- **Secure Storage**: Environment variables safeguard sensitive information.
- **Ease of Use**: Allows seamless API access throughout the project.

This step ensures secure and authenticated interactions with the OpenAI API.


In [2]:
# Step 2: Setting up OpenAI key
# Note: You need an OpenAI API key to proceed. Set it below.
openai.api_key = os.environ["OPENAI_API_KEY"]


# Step 3: Generating Car Data

We simulate a real-world dataset by creating a collection of car descriptions. This dataset includes key fields like name, price, engine type, and description.

### Why Generate Car Data?
1. **Data for Embeddings**: Provides input for creating vector embeddings.
2. **Simulated Real-World Scenario**: Demonstrates how the system works with structured data.

### Workflow
- Manually define a small dataset of cars with fields such as `name`, `price`, and `description`.
- Use this dataset as the foundation for embeddings and retrieval.

### Code Highlights
- **Data Structure**: JSON-like format captures relevant car details.
- **Dataset Flexibility**: Easy to expand for larger-scale applications.

This step provides the initial dataset required for embedding storage and retrieval.


In [4]:
# Step 3: Generate data for the cars
"""
To simulate a real dataset, we'll generate information about cars. Each car will have a name, price, engine type, and description.
The following data is used to showcase how information is stored and used in the vector database for retrieval.
"""

# Here we define a small set of initial car data manually.
# This will serve as our base dataset to which we will later add more generated cars.
cars = [
    {
        "name": "Toyota Corolla 2024",
        "price": "$25,000",
        "engine": "1.8L Inline-4",
        "year": 2024,
        "country": "Japan",
        "manufacturer": "Toyota",
        "description": "The Toyota Corolla 2024 offers reliability, fuel efficiency, and a comfortable ride, making it an ideal choice for families and urban commuters."
    },
    {
        "name": "Ford Mustang GT 2024",
        "price": "$45,000",
        "engine": "5.0L V8",
        "year": 2024,
        "country": "United States",
        "manufacturer": "Ford",
        "description": "The Ford Mustang GT 2024 delivers exhilarating performance with its powerful V8 engine, sporty handling, and iconic design."
    },
    {
        "name": "BMW 3 Series 2024",
        "price": "$42,000",
        "engine": "2.0L Turbo Inline-4",
        "year": 2024,
        "country": "Germany",
        "manufacturer": "BMW",
        "description": "The BMW 3 Series 2024 is a luxury sedan offering a perfect balance of performance, style, and advanced technology."
    },
    {
        "name": "Hyundai Ioniq 5 2024",
        "price": "$39,000",
        "engine": "Electric",
        "year": 2024,
        "country": "South Korea",
        "manufacturer": "Hyundai",
        "description": "The Hyundai Ioniq 5 2024 is a fully electric SUV with a futuristic design, exceptional range, and innovative features."
    },
]


In [4]:
# Step 3a: Generate more car data using GPT-3
"""
To enhance the dataset, we will generate additional car entries using OpenAI's GPT-3 model.
The generated data will include car name, price, engine type, and a detailed description.
We will use a prompt to guide the model to return data in a consistent and predictable format.
"""

"\nTo enhance the dataset, we will generate additional car entries using OpenAI's GPT-3 model.\nThe generated data will include car name, price, engine type, and a detailed description.\nWe will use a prompt to guide the model to return data in a consistent and predictable format.\n"

# Step 3a: Expanding the Dataset Using GPT-3

To enhance the dataset, we dynamically generate more car data using GPT-3, ensuring variety and uniqueness.

### Why Use GPT-3 for Data Generation?
1. **Scalability**: Automates the creation of large datasets.
2. **Diversity**: Produces varied and realistic data entries.

### Workflow
- Prompt GPT-3 with specific instructions to generate new car entries.
- Ensure unique entries by checking against existing data.

### Code Highlights
- **Prompt Engineering**: Guides GPT-3 to produce consistent and structured output.
- **Duplicate Checks**: Ensures no repeated entries in the dataset.

This step expands the dataset, enabling more robust testing and retrieval.


In [5]:
def generate_car_data(num_cars=50, existing_names=None):
    """
    Generates car data using OpenAI's GPT-3 model.
    Args:
        num_cars (int): Number of car entries to generate.
    Returns:
        List[dict]: A list of dictionaries containing car details.
    """
    if not existing_names:
        existing_names = set()
    car_data = []
    while len(car_data) < num_cars:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": (
                    "You are an expert assistant tasked with creating unique and realistic car entries. "
                    "Each car entry must have a unique name and be described in detail."
                    "Create cars for countries like Japan, Germany, South Korea, and the United States."
                )},
                {"role": "user", "content": (
                    "Generate details for a car including the following fields: \n"
                    "Name: <Unique Car Name using creative names>\n"
                    "Price: <Car Price>\n"
                    "Engine: <Engine Type with details if electric, combustion or hibrid.>\n"
                    "Year: <Manufacturing Year>\n"
                    "Country: <Country of Origin>\n"
                    "Manufacturer: <Car Manufacturing Company>\n"
                    "Description: <Detailed description of the Car>\n"
                    "Ensure that the car name is not a duplicate and provide realistic and varied entries."
                )}
            ],
            max_tokens=200
        )
        car_details = response['choices'][0]['message']['content'].strip().split('\n')
        try:
            car = {
                "name": car_details[0].split(': ')[1],
                "price": car_details[1].split(': ')[1],
                "engine": car_details[2].split(': ')[1],
                "year": int(car_details[3].split(': ')[1]),
                "country": car_details[4].split(': ')[1],
                "manufacturer": car_details[5].split(': ')[1],
                "description": car_details[6].split(': ')[1]
            }
            if car['name'] not in existing_names:
                existing_names.add(car['name'])
                car_data.append(car)
            else:
                print(f"Duplicate car detected: {car['name']}. Skipping entry. Current Number of cars: {len(car_data)}.")
        except IndexError:
            continue
    return car_data

# Append the generated cars to the existing cars list
"""
Here, we append the 26 generated cars to our initial dataset.
This results in a dataset of 30 cars that will be used for embedding storage and retrieval.
"""
cars += generate_car_data(26)


Duplicate car detected: Zenith GT. Skipping entry. Current Number of cars: 8.
Duplicate car detected: Zenith GT. Skipping entry. Current Number of cars: 8.
Duplicate car detected: Zenith Zephyr. Skipping entry. Current Number of cars: 8.
Duplicate car detected: Zenith GT. Skipping entry. Current Number of cars: 19.
Duplicate car detected: Zenith Zephyr. Skipping entry. Current Number of cars: 22.


In [7]:
cars[-5:]

[{'name': 'Sakura Blossom',
  'price': '$35,000',
  'engine': 'Hybrid engine - Combustion engine combined with electric motor',
  'year': 2022,
  'country': 'Japan',
  'manufacturer': 'Toyota',
  'description': "The Sakura Blossom is a sleek and eco-friendly hybrid sedan designed by Toyota, inspired by the beauty of Japan's cherry blossoms. Equipped with a powerful yet fuel-efficient hybrid engine, this car offers a smooth and quiet ride, perfect for city commuting and long road trips. The interior features premium materials, advanced technology, and spacious seating for ultimate comfort. With its striking design and advanced features, the Sakura Blossom is a symbol of innovation and elegance."},
 {'name': 'Thunderhawk GT',
  'price': '$60,000',
  'engine': '3.0L V6 twin-turbo',
  'year': 2022,
  'country': 'United States',
  'manufacturer': 'Thunder Motors',
  'description': 'The Thunderhawk GT is a sleek and powerful American sports car that combines luxury with performance. Its 3.0L

# Step 4: Initializing ChromaDB and Preparing Embeddings

This step involves setting up ChromaDB, an in-memory vector database, and generating embeddings for the car descriptions.

### Why Use ChromaDB?
1. **Efficient Storage**: Stores vector embeddings for fast retrieval.
2. **Scalable Queries**: Handles similarity-based searches effectively.

### Workflow
- Initialize a ChromaDB client and create a collection for car embeddings.
- Convert car descriptions into embeddings using OpenAI’s models.

### Code Highlights
- **Embedding Generation**: Leverages OpenAI’s `text-embedding-ada-002` model.
- **Database Integration**: Uses ChromaDB for vector storage and retrieval.

This step prepares the system for embedding-based similarity searches.


In [15]:
# Step 4: Initialize Chroma DB and prepare embeddings
"""
We will use OpenAI embeddings to convert the car descriptions into vectors, which can be easily stored and queried in Chroma.
Chroma will serve as our in-memory vector database, allowing us to perform fast similarity searches for the car descriptions.
"""

# Set up Chroma and OpenAI embedding function
client = chromadb.Client()
openai_ef = embedding_functions.OpenAIEmbeddingFunction(api_key=openai.api_key, 
                                                        model_name="text-embedding-ada-002")

# Create a new collection for storing car embeddings
"""
A Chroma collection is similar to a table in a database.
In this collection, we will store embeddings representing each car's description along with metadata such as car name, price, and engine type.
"""
car_collection = client.create_collection(name="cars_collection", embedding_function=openai_ef, metadata={"hnsw:space": "cosine"})

# Add car data to the Chroma collection
car_ids = [str(i) for i in range(len(cars))]
car_descriptions = [car["description"] for car in cars]
car_metadata = [{
    "name": car["name"],
    "price": car["price"],
    "engine": car["engine"],
    "year": car["year"],
    "country": car["country"],
    "manufacturer": car["manufacturer"]
} for car in cars]


"""
Adding the car data to Chroma involves specifying unique IDs for each car, the descriptions to embed, and relevant metadata.
The metadata will be useful for presenting information to the user when we query the database.
"""
car_collection.add(ids=car_ids, metadatas=car_metadata, documents=car_descriptions)


In [16]:
# Display all data in the collection as a pandas DataFrame
"""
We will now retrieve all metadata from the Chroma collection and display it as a pandas DataFrame.
This provides a clear view of the data stored in the collection.
"""
# Retrieve all metadata, embeddings, and documents (descriptions) from the Chroma collection
all_metadata = car_collection.get(ids=car_ids, include=["embeddings", "documents", "metadatas"])

# Create a DataFrame from the metadata
car_data = pd.DataFrame(all_metadata['metadatas'])
car_data["description"] = all_metadata['documents']
car_data["embedding"] = all_metadata['embeddings'] 
print("\Sample 5 Cars Entry in the Collection:")
car_data.sample(n=5)

\Sample 5 Cars Entry in the Collection:


,country,engine,manufacturer,name,price,year,description,embedding
10,United States,Hybrid engine - Gasoline and Electric,Nova Motors,Solaris GT,"$45,000",2022,The Solaris GT is a sleek and eco-friendly hyb...,"[0.0057664294727146626, 0.0033361271489411592,..."
2,Japan,Hybrid engine - combines a 2.0-liter turbochar...,Zenith Motors,Zenith Xeno,"$45,000",2023,The Zenith Xeno is a futuristic hybrid sports ...,"[0.008428807370364666, 0.008408849127590656, 0..."
18,Japan,Hybrid engine - Combustion engine combined wit...,Toyota,Sakura Blossom,"$35,000",2022,The Sakura Blossom is a sleek and eco-friendly...,"[0.002476014196872711, 0.005401615984737873, -..."
20,Japan,Electric,Sakura Motors,Sakura GT7,"$45,000",2023,The Sakura GT7 is a sleek and futuristic elect...,"[0.009560051374137402, 0.032799847424030304, -..."
6,Japan,Electric,Yokai Motors,Zenith X1,"$45,000",2023,The Zenith X1 is a futuristic all-electric sed...,"[0.017563944682478905, 0.011302477680146694, 0..."


# Step 5: Querying ChromaDB for Information

This step demonstrates querying the ChromaDB collection to retrieve relevant cars based on user queries.

### Why Query ChromaDB?
1. **Retrieve Relevant Data**: Finds the most similar entries to a given query.
2. **Natural Language Interface**: Allows users to search using everyday language.

### Workflow
- Use natural language prompts to query the vector database.
- Retrieve and display relevant metadata and descriptions.

### Code Highlights
- **Similarity Search**: Retrieves the top matches based on vector embeddings.
- **Flexible Queries**: Handles diverse user inputs effectively.

This step showcases the retrieval capabilities of the RAG system.


In [20]:
# Step 5: Querying Chroma for information
"""
In this step, we will demonstrate how to query the Chroma collection to find relevant cars.
We will use a natural language prompt to find cars that match specific requirements.
The embeddings will allow us to determine the similarity between the query and the car descriptions.
"""

# Example query prompt
prompts = [
    "I want a comfortable car for my family with good safety features.",
    "Looking for a sporty car with high speed and acceleration.",
    "Find me an eco-friendly electric car for urban use in the United States.",
    "I need a luxury sedan with a hybrid engine for a comfortable commute.",
    "Suggest an affordable car with great fuel efficiency for a student."
]

prompt = np.random.choice(prompts)
print(f"The user's prompt is: {prompt}")

# Retrieve the top match from Chroma collection
"""
Using the `query` method, we search for the most relevant car based on the given prompt.
The query will return the car description that is most similar to the provided input.
"""
results = car_collection.query(query_texts=[prompt], n_results=1)

# Display the result
"""
Once we get the result, we extract the metadata for the recommended car, such as its name, price, and engine type.
We then print out the recommended car's details for the user.
"""
result_metadata = results["metadatas"][0][0]
result_name = result_metadata["name"]
result_price = result_metadata["price"]
result_engine = result_metadata["engine"]
result_year = result_metadata["year"]
result_manufacture = result_metadata["manufacturer"]
result_country = result_metadata["country"]
result_description = results['documents']
result_score = results['distances'][0][0]

print(f"Recommended Car: {result_name} with Consine Similarity: {result_score:.4f}\nPrice: {result_price}\nEngine: {result_engine}\nYear: {result_year}\nManufacturer: {result_manufacture}\nCountry: {result_country}\nDescription: {result_description[0][0]}")

"""
This result demonstrates how the RAG approach, combining embeddings and natural language queries, can provide users with relevant and personalized insights.
For a user looking for a specific type of car, such as one that is comfortable for a family, our system can find the best match from the dataset.
"""


The user's prompt is: Suggest an affordable car with great fuel efficiency for a student.
Recommended Car: Toyota Corolla 2024 with Consine Similarity: 0.1623
Price: $25,000
Engine: 1.8L Inline-4
Year: 2024
Manufacturer: Toyota
Country: Japan
Description: The Toyota Corolla 2024 offers reliability, fuel efficiency, and a comfortable ride, making it an ideal choice for families and urban commuters.


'\nThis result demonstrates how the RAG approach, combining embeddings and natural language queries, can provide users with relevant and personalized insights.\nFor a user looking for a specific type of car, such as one that is comfortable for a family, our system can find the best match from the dataset.\n'

In [27]:
# Additional query for metadata (Country-specific search)
"""
In this step, we demonstrate querying not only for text similarity but also filtering results based on metadata like country of origin.
"""
# Specify the country filter
country_filter = "United States"  # Replace with the desired country

# Query the Chroma collection with the country filter
results = car_collection.query(
    query_texts=[prompt],
    n_results=3,
    where={"country": country_filter}  # Metadata filter for country
)

# Display results with country filtering and include the description
print(f"The user's prompt is: {prompt}")
for idx, metadata in enumerate(results["metadatas"][0]):
    name = metadata["name"]
    price = metadata["price"]
    engine = metadata["engine"]
    year = metadata["year"]
    country = metadata["country"]
    manufacturer = metadata["manufacturer"]
    description = results["documents"][0][idx]  # Extract the corresponding description
    score = results["distances"][0][idx]

    print(f"\nResult {idx + 1}:")
    print(f"Name: {name} - Cosine Similarity: {score:.4f}")
    print(f"Price: {price}")
    print(f"Engine: {engine}")
    print(f"Year: {year}")
    print(f"Country: {country}")
    print(f"Manufacturer: {manufacturer}")
    print(f"Description: {description}\n")

The user's prompt is: Suggest an affordable car with great fuel efficiency for a student.

Result 1:
Name: Solaris GT - Cosine Similarity: 0.2043
Price: $45,000
Engine: Hybrid engine - Gasoline and Electric
Year: 2022
Country: United States
Manufacturer: Nova Motors
Description: The Solaris GT is a sleek and eco-friendly hybrid sports car that offers a perfect blend of performance and sustainability. Its gasoline-electric hybrid engine delivers powerful acceleration while reducing emissions. The futuristic design features aerodynamic curves and LED headlights, creating a bold and stylish appearance. The interior is equipped with cutting-edge technology, including a touchscreen infotainment system and advanced safety features. The Solaris GT is the perfect choice for drivers who prioritize both performance and environmental consciousness.


Result 2:
Name: Zenith Zephyr - Cosine Similarity: 0.2129
Price: $45,000
Engine: Hybrid - 2.0L turbocharged inline-4 + electric motor
Year: 2022
Cou

# Step 7: Enriching the Output Using an LLM

In this step, we enhance the user experience by generating a personalized, human-like message for the car buyer. This message is crafted by leveraging the retrieved car metadata and combining it with GPT-3.5's natural language capabilities.

### Why Enrich the Output?
1. **Personalized Experience**: Tailored messages make the recommendations more engaging and user-specific.
2. **Improved Communication**: Highlights key features of the car in an appealing way.
3. **Contextual Relevance**: Aligns the message with user preferences and needs.

### Workflow
- Extract car metadata such as `name`, `price`, `engine`, `year`, `manufacturer`, and `country`.
- Pass the metadata to GPT-3.5 to generate a friendly and engaging recommendation message.
- Display the enriched message to the user.

### Code Highlights
- **Metadata Integration**: Uses car details to craft a personalized message.
- **LLM Usage**: GPT-3.5 generates context-aware and user-friendly output.
- **Dynamic Messaging**: Messages are adaptable to different cars and user queries.

This step demonstrates how to combine retrieval with generation to create a more interactive and valuable user experience.


In [31]:
# Step 6: Enriching the output using an LLM
"""
We will now use the LLM to generate a personalized message to the car buyer, leveraging the retrieved car's metadata.
This step showcases how to combine retrieval with generation for a more tailored experience.
"""
def generate_message_for_buyer(car_name, car_price, car_engine, car_year, car_manufacturer, car_country, prompt):
    """
    Generates a personalized message for the car buyer.
    Args:
        car_name (str): Name of the recommended car.
        car_price (str): Price of the recommended car.
        car_engine (str): Engine type of the recommended car.
        car_year (str): Year of the recommended car.
        car_manufacturer (str): Manufacturer of the recommended car.
        car_country (str): Country of origin of the recommended car.
        prompt (str): Contextual information about the buyer.
    Returns:
        str: A personalized message for the buyer.
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant to sell a car."},
            {"role": "user", "content": (
                f"Create a friendly and engaging message for a car buyer who wants to buy a car with this context {prompt} interested in the {car_name}. \n"
                f"The user wants a car from a give {car_country} and add relevant information about this.\n"
                f"Mention its price of {car_price} and highlight the {car_engine} engine's benefits, the performance, and other key features.\n"
                f"Highlight the Year {car_year} and the Car Producer/Manufacturer {car_manufacturer} as key points."
            )}
        ],
        max_tokens=300
    )
    return response['choices'][0]['message']['content'].strip()

# Generate the enriched message
enriched_message = generate_message_for_buyer(name, 
                                              price, 
                                              engine, 
                                              year, 
                                              manufacturer,
                                              country,
                                              prompt
                                            )
print("\nPersonalized Message:")
print(enriched_message)

"""
This step demonstrates how to use the retrieved car data to generate an enriched, human-like response that enhances user engagement.
The message can be further personalized based on additional user preferences or context.
"""


Personalized Message:
🚗 Looking for an affordable and fuel-efficient car that perfectly suits your student lifestyle? Let me introduce you to the Zenith Electra! 🌟

This stunning electric car from Zenith Motors not only looks sleek and stylish but also comes with amazing features that make it a great choice for students. Priced at just $45,000, the Zenith Electra offers incredible value for its quality and performance. 

With its electric engine, you'll enjoy the benefits of eco-friendly driving while saving money on fuel costs. The Zenith Electra's performance is top-notch, delivering a smooth and quiet ride that's perfect for navigating city streets or cruising on the highway.

The 2022 model of the Zenith Electra showcases the latest in electric vehicle technology, ensuring you have a reliable and efficient car to get you where you need to go. Plus, being made in the United States, you can trust in the quality and craftsmanship that comes with a Zenith Motors vehicle.

Don't miss o

'\nThis step demonstrates how to use the retrieved car data to generate an enriched, human-like response that enhances user engagement.\nThe message can be further personalized based on additional user preferences or context.\n'

# Step 7a: Generating Visual Representations for Cars

In this step, we enhance the user experience by generating realistic images of the recommended cars using an AI image generation API, such as OpenAI's DALL-E. This visual representation complements the textual description, making the results more engaging and tangible for users.

### Why Generate Images?
1. **Enhanced User Experience**: Visuals provide a clearer understanding of the recommendation.
2. **Personalization**: Images can reflect specific user queries, like model, year, or manufacturer.
3. **Engagement**: Combining text and visuals creates a richer and more interactive system.

### Workflow
- Create a descriptive image prompt using the car's name, year, and manufacturer.
- Use the API to generate a high-quality, realistic image based on the prompt.
- Display the generated image alongside the recommendation.

### Code Highlights
- **Prompt Engineering**: Ensures the generated image aligns with the car's details.
- **API Integration**: Uses OpenAI's `Image.create` method for image generation.
- **Realistic Details**: Includes elements like showroom setting, side angles, and daylight effects.

This step demonstrates how visual representations can complement the RAG system, adding depth and engagement to user interactions.


In [34]:
def generate_image_for_car(car_name, car_year, car_manufacturer):
    """
    Generates an image for the recommended car using DALL-E or another image generation API.
    Args:
        car_name (str): Name of the car.
        car_year (str): Year of the car.
        car_manufacturer (str): Manufacturer of the car.
    Returns:
        str: File path or URL of the generated image.
    """
    image_prompt = (
        f"A full exterior view of a car from year {car_year}, manufacturer {car_manufacturer} and name {car_name} parked in a showroom. "
        "Show the car from the side angle, including wheels, headlights, and body details. "
        "The image should depict a realistic, high-quality photo in natural daylight."
        f"Add a plate with the the car price {car_year} and the car name {car_name}"
    )
    response = openai.Image.create(
        prompt=image_prompt,
        n=1,
        size="512x512"  # Specify the smaller image size
    )
    return response["data"][0]["url"]

print("\nPersonalized Message:")
print(enriched_message)

# Generate the car image
image_url = generate_image_for_car(name, year, manufacturer)

print("\nGenerated Image:")
display(Image(url=image_url))

"""
This step demonstrates how to use the retrieved car data to generate both an enriched, human-like response and a visual representation of the car, enhancing user engagement.
The message and image can be further personalized based on additional user preferences or context.
"""


Personalized Message:
🚗 Looking for an affordable and fuel-efficient car that perfectly suits your student lifestyle? Let me introduce you to the Zenith Electra! 🌟

This stunning electric car from Zenith Motors not only looks sleek and stylish but also comes with amazing features that make it a great choice for students. Priced at just $45,000, the Zenith Electra offers incredible value for its quality and performance. 

With its electric engine, you'll enjoy the benefits of eco-friendly driving while saving money on fuel costs. The Zenith Electra's performance is top-notch, delivering a smooth and quiet ride that's perfect for navigating city streets or cruising on the highway.

The 2022 model of the Zenith Electra showcases the latest in electric vehicle technology, ensuring you have a reliable and efficient car to get you where you need to go. Plus, being made in the United States, you can trust in the quality and craftsmanship that comes with a Zenith Motors vehicle.

Don't miss o

'\nThis step demonstrates how to use the retrieved car data to generate both an enriched, human-like response and a visual representation of the car, enhancing user engagement.\nThe message and image can be further personalized based on additional user preferences or context.\n'

# Step 8: Reranking with LLMs

Reranking is a critical component in ensuring the most relevant results are presented to users. This step introduces a robust retry mechanism that enhances the reliability of the reranking process when utilizing GPT-3.5 for scoring relevance. The retry mechanism ensures consistent results even in cases where API calls fail due to network or token errors.

### Why Focus on the Reranking Mechanism?
1. **Reliability Under Stress**: Handles transient errors from GPT-3.5 API calls with up to three retry attempts.
2. **Contextual Accuracy**: Combines GPT-derived relevance scores with vector similarity scores for improved ranking precision.
3. **Streamlined Results**: Logs skipped entries gracefully to maintain overall system performance and user experience.

### Reranking Workflow
- Query results from ChromaDB are passed to GPT-3.5 for relevance scoring.
- If an error occurs during a GPT call, the retry mechanism attempts up to three calls.
- Results with successful scores are combined with vector similarity scores to calculate a weighted ranking.
- Entries that fail after retries are logged and skipped.

### Code Highlights
- **Retry Logic**: Built into the `rerank_gpt_call` function, ensuring a seamless fallback for errors.
- **Combined Scoring**: Weighted scores from GPT relevance and vector similarity ensure robust ranking.
- **Error Logging**: Keeps track of skipped results, enabling post-analysis for improvements.

This enhanced reranking mechanism underpins the robustness of the RAG system, ensuring meaningful, context-aware outputs even under challenging conditions.


In [35]:
def rerank_gpt_call(query, description, temperature=0.3):
    """
    Call GPT-3.5 to score the relevance of a car description to a query.
    Args:
        query (str): The original user query.
        description (str): The description of a car.
        temperature (float): The creativity level for the response (default is 0.3 for more deterministic results).
    Returns:
        int: The score.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are an AI ranking assistant. Your task is to evaluate the relevance of a car description "
                        "to a given user query. Focus on matching keywords, themes, and context."
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"Query: \"{query}\"\n"
                        f"Car Description: \"{description}\"\n"
                        "On a scale from 1 to 10, where:\n"
                        "- 1 means 'completely irrelevant',\n"
                        "- 5 means 'somewhat relevant', and\n"
                        "- 10 means 'perfectly relevant',\n"
                        "rate the relevance of the car description to the query and provide only the numeric score."
                    )
                }
            ],
            temperature=temperature,  # Adding temperature control
            max_tokens=10  # Ensuring minimal tokens for numeric output
        )
        llm_score = int(response['choices'][0]['message']['content'].strip())  # Extracting score from response
        return llm_score
    except Exception as e:
        print(f"Error during GPT call: {e}")
        return None


def rerank_results(query, results):
    """
    Re-rank the results based on additional contextual evaluation.
    Args:
        query (str): The original user query.
        results (dict): The results returned by ChromaDB query.
    Returns:
        list: Re-ranked results with metadata and scores.
    """
    reranked_results = []
    for idx, metadata in enumerate(results["metadatas"][0]):
        description = results["documents"][0][idx]
        score = results["distances"][0][idx]

        llm_score = None
        attempts = 0
        while llm_score is None and attempts < 3:
            attempts += 1
            llm_score = rerank_gpt_call(query, description, temperature=0.5)
            if llm_score is None:
                print(f"Retrying GPT call for result {idx + 1}, attempt {attempts}.")

        if llm_score is None:
            print(f"Failed to process result {idx + 1} after {attempts} attempts. Skipping.")
            continue

        # Combine Chroma score and LLM score
        combined_score = score * 0.5 + llm_score * 0.5  # Adjust weights as needed
        metadata["combined_score"] = combined_score
        reranked_results.append((metadata, combined_score))

    # Sort results by combined score
    reranked_results = sorted(reranked_results, key=lambda x: x[1], reverse=True)
    return reranked_results

# Example usage of the reranker
# Apply reranker to the results
reranked = rerank_results(prompt, results)
print(f"The user's prompt is: {prompt}")
print("Re-ranked Results:")
for idx, (metadata, score) in enumerate(reranked):
    print(f"\nRank {idx + 1}:")
    print(f"Name: {metadata['name']}")
    print(f"Score: {score}")
    print(f"Details: {metadata}")


The user's prompt is: Suggest an affordable car with great fuel efficiency for a student.
Re-ranked Results:

Rank 1:
Name: Zenith Zephyr
Score: 4.106460303068161
Details: {'country': 'United States', 'engine': 'Hybrid - 2.0L turbocharged inline-4 + electric motor', 'manufacturer': 'Zenith Motors', 'name': 'Zenith Zephyr', 'price': '$45,000', 'year': 2022, 'combined_score': 4.106460303068161}

Rank 2:
Name: Solaris GT
Score: 1.6021298468112946
Details: {'country': 'United States', 'engine': 'Hybrid engine - Gasoline and Electric', 'manufacturer': 'Nova Motors', 'name': 'Solaris GT', 'price': '$45,000', 'year': 2022, 'combined_score': 1.6021298468112946}

Rank 3:
Name: Zenith Electra
Score: 1.109113872051239
Details: {'country': 'United States', 'engine': 'Electric', 'manufacturer': 'Zenith Motors', 'name': 'Zenith Electra', 'price': '$45,000', 'year': 2022, 'combined_score': 1.109113872051239}


In [37]:
reranked_result_name = reranked[0][0]["name"]
reranked_result_price = reranked[0][0]["price"]
reranked_result_engine = reranked[0][0]["engine"]
reranked_result_year = reranked[0][0]["year"]
reranked_result_manufacture = reranked[0][0]["manufacturer"]
reranked_result_country = reranked[0][0]["country"]


# Generate the enriched message
enriched_message = generate_message_for_buyer(reranked_result_name, 
                                              reranked_result_price, 
                                              reranked_result_engine, 
                                              reranked_result_year, 
                                              reranked_result_manufacture,
                                              reranked_result_country,
                                              prompt=prompt
                                            )
print("\nReranked Personalized Message:")
print(enriched_message)

# Generate the car image
reranked_image_url = generate_image_for_car(reranked_result_name, reranked_result_year, reranked_result_manufacture)

print("\nGenerated Image:")
display(Image(url=reranked_image_url))


Reranked Personalized Message:
🚗 Hello there! Looking for an affordable yet fuel-efficient car for your student life in the United States? Let me introduce you to the fantastic Zenith Zephyr, a perfect blend of style, efficiency, and performance!

🌟 This sleek ride from 2022 comes at a price of $45,000, making it an excellent choice for students looking for a reliable and cost-effective vehicle. Powered by a Hybrid - 2.0L turbocharged inline-4 + electric motor engine, the Zenith Zephyr offers the best of both worlds - impressive fuel efficiency and a thrilling driving experience.

⚡️ With its hybrid engine, you'll enjoy not only great gas mileage but also reduced emissions, making it an eco-friendly choice that suits your student lifestyle. Plus, the Zenith Zephyr's excellent performance capabilities ensure a smooth and enjoyable ride wherever your adventures take you.

🔑 Key features of the Zenith Zephyr include advanced safety options, modern technology integration for a connected d